In [4]:
import torch
import nltk
import numpy as np
import pandas as pd
import gzip
import torch
import requests
import json
from nltk.corpus import brown
from torch.utils.data import Dataset, DataLoader
import random
import io
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from collections import Counter
import requests

download and prepare the Brown corpus from NLTK for training a Word2Vec model, which is used for creating embeddings of words.

Download Squad2 dataset, split it into training and testing sets, and save them to files.

In [ ]:
# Download SQuAD2 dataset
url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json"
response = requests.get(url)

#Save the dataset to a file
dataset_file = "squad2.json"
with open(dataset_file, 'wb') as f:
   f.write(response.content)

#Split dataset into train and test files
train_ratio = 0.8
train_file = "train.json"
test_file = "test.json"

with open(dataset_file, 'r') as f:
    squad_data = json.load(f)
    data = squad_data['data']

    # Shuffle the data
    random.shuffle(data)

    # Split into train and test sets
    split_index = int(len(data) * train_ratio)
    train_data = data[:split_index]
    test_data = data[split_index:]

    # Save train data to file
    train_squad_data = {'data': train_data}
    with open(train_file, 'w') as train_f:
        json.dump(train_squad_data, train_f)

    # Save test data to file
    test_squad_data = {'data': test_data}
    with open(test_file, 'w') as test_f:
        json.dump(test_squad_data, test_f)

In [5]:
#We will use this function on each file in the dataset - test and train
#only the filename is required as the previous function returns the specified file to the same location

def extract_squad_data(file_name):
    with open(file_name, 'r', encoding='utf-8') as f:
        file_name = json.load(f)

    paragraphs = []
    for article in file_name['data']:
        for paragraph in article['paragraphs']:
            for qa in paragraph['qas']:
                question = qa['question']
                answers = [answer['text'] for answer in qa['answers']]
                for answer in answers:
                    paragraphs.append((question, answer))

    df = pd.DataFrame(paragraphs, columns=['Question', 'Answer'])
    return df

In [6]:
#convert .json files to dataframes
train_df = extract_squad_data("train.json") 
test_df = extract_squad_data("test.json")


In [7]:


import pandas as pd
import nltk
import string
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer("english")

def clean_text(df):
    cleaned_df = df.copy()
    for column in cleaned_df:
        cleaned_df[column] = cleaned_df[column].apply(lambda sentence: process_sentence(sentence))
    return cleaned_df

def process_sentence(sentence):
    sentence = ''.join([s.lower() for s in sentence if s not in string.punctuation])
    sentence = ' '.join(stemmer.stem(w) for w in sentence.split())
    tokens = nltk.tokenize.RegexpTokenizer(r'\w+').tokenize(sentence)

    return ' '.join(tokens) 
# usage:
# df = clean_text(df)


In [8]:
train_clean_df = clean_text(train_df.head(50))


In [49]:
print(train_clean_df)

                                            Question         Answer
0  what citi in the unit state has the highest popul       new york
1               in what citi is the unit nation base       new york
2  what citi has been call the cultur capit of th...       new york
3  what american citi welcom the largest number o...       new york
4  the major gateway for immigr has been which us...  new york citi
5  the most popul citi in the unit state is which...  new york citi
6             how mani borough compris new york citi           five
7  in what year were the five borough combin into...           1898
8  in 2014 what did the census estim the popul of...        8491079
9    what is the size of new york citi in squar mile            305


In [10]:
class Vocab:
    def __init__(self):
        self.word2index = {}
        self.index2word = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 4: "<UNK>"}
        self.word2count = {}
        self.n_words = 4  # Count <PAD>, <SOS>, <EOS>, <UNK>
    def build_vocab(self, sentences):
        for sentence in sentences:
            for word in self.tokenize(sentence):
                self.add_word(word)
    def tokenize(self, sentence):
        return sentence.strip().split()
    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.word2count[word] = 1
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [11]:
question_vocab = Vocab()
answer_vocab = Vocab()

questions = train_clean_df['Question'].tolist()
answers = train_clean_df['Answer'].tolist()

question_vocab.build_vocab(questions)
answer_vocab.build_vocab(answers)

In [12]:
print(question_vocab.index2word)

{0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 4: 'what', 5: 'is', 6: 'a', 7: 'sourc', 8: 'for', 9: 'the', 10: 'particl', 11: 'in', 12: 'paper', 13: 'common', 14: 'use', 15: 'last', 16: 'step', 17: 'product', 18: 'when', 19: 'was', 20: 'pulp', 21: 'papermak', 22: 'process', 23: 'develop', 24: 'countri', 25: 'lead', 26: 'of', 27: 'second', 28: 'largest', 29: 'produc', 30: 'which', 31: 'court', 32: 'eunuch', 33: 'associ', 34: 'with', 35: 'who', 36: 'credit', 37: 'materi', 38: 'did', 39: 'replac', 40: 'upon', 41: 'it', 42: 'invent', 43: 'age', 44: 'contribut', 45: 'toward', 46: 'centuri', 47: 'first', 48: 'attribut', 49: 'to', 50: 'spread', 51: 'from', 52: 'china', 53: 'europ', 54: 'middl', 55: 'eastern', 56: 'citi', 57: 'name', 58: 'take', 59: 'on', 60: 'baghdad', 61: 'becom', 62: 'much', 63: 'cheaper', 64: 'than', 65: 'befor', 66: 'canadian', 67: 'inventor', 68: 'help', 69: 'fg', 70: 'keller', 71: 'their', 72: 'own', 73: 'latin', 74: 'word', 75: 'deriv', 76: 'languag', 77: 'papyrus', 78: 'chang',

In [13]:
def pad_or_cut(df, column_names, length):
    pad_symbol = "<PAD>"
    
    for column_name in column_names:
        df[column_name] = df[column_name].apply(lambda x: (x.split()[:length] + [pad_symbol]*length)[:length])
        df[column_name] = df[column_name].apply(lambda x: ' '.join(x))
    return df


In [14]:
padded_df = pad_or_cut(train_clean_df, ['Question', 'Answer'], 10)

In [15]:
padded_df.head()

,Question,Answer
0,what is a sourc for the particl in paper <PAD>,wood <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>...
1,what is a common use for paper <PAD> <PAD> <PAD>,write <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...
2,what is the last step in paper product <PAD> <...,dri <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> ...
3,when was the pulp papermak process develop <PA...,2nd centuri ad <PAD> <PAD> <PAD> <PAD> <PAD> <...
4,what countri was the process develop in <PAD> ...,china <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD...


In [16]:
def words_to_indices(df, question_vocab, answer_vocab):
    df['Question'] = df['Question'].apply(lambda sentence: [question_vocab.word2index[word] if word in question_vocab.word2index else question_vocab.word2index['<UNK>'] for word in sentence.split()])
    df['Answer'] = df['Answer'].apply(lambda sentence: [answer_vocab.word2index[word] if word in answer_vocab.word2index else answer_vocab.word2index['<UNK>'] for word in sentence.split()])
    return df

In [17]:
indices_df = words_to_indices(padded_df, question_vocab, answer_vocab)

KeyError: '<UNK>'

In [75]:
indices_df.head()

,Question,Answer
0,"[4, 5, 6, 7, 8, 9, 10, 7, 11, 12]","[4, 5, 6, 6, 6, 6, 6, 6, 6, 6]"
1,"[6, 4, 5, 13, 7, 8, 14, 15, 16, 16]","[4, 5, 6, 6, 6, 6, 6, 6, 6, 6]"
2,"[4, 5, 10, 17, 18, 7, 19, 20, 21, 7]","[4, 5, 6, 6, 6, 6, 6, 6, 6, 6]"
3,"[4, 22, 5, 23, 7, 24, 25, 21, 26, 27]","[4, 5, 6, 6, 6, 6, 6, 6, 6, 6]"
4,"[7, 28, 29, 30, 27, 10, 17, 31, 32, 5]","[4, 5, 7, 6, 6, 6, 6, 6, 6, 6]"
